In [ ]:
import pandas as pd
import os, requests, json, re
import openai
import requests
from datetime import datetime
from pytz import timezone
import torch

In [ ]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_dataset

# Task 1: From User Input we classify the Intent (Task) that is whether to book cab, order food, book movie tickets etc.

In [ ]:
# Read the CSV and map labels
df = pd.read_csv("synthetic_intent_dataset_v2.csv")

df.loc[df["label"]==1, "label"] = 0
df.loc[df["label"]==2, "label"] = 1
df.loc[df["label"]==7, "label"] = 2
df_in_context = df.copy()

In [ ]:
# Shuffle with a fixed seed for reproducibility
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into Train and Test (70 : 30)
train_size = int(0.7 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

# Save Train and Test Splits as CSV
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [ ]:
# Load the Splits Using `load_dataset`
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

display(dataset)

In [ ]:
# Distribution of labels
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Extract texts and labels from the dataset
train_texts = [item["text"] for item in dataset["train"]]
train_labels = [item["label"] for item in dataset["train"]]

test_texts = [item["text"] for item in dataset["test"]]
test_labels = [item["label"] for item in dataset["test"]]

# Count label frequencies
label_counter = Counter(train_labels)

# Create a DataFrame from label frequencies
df = pd.DataFrame.from_dict(label_counter, orient="index", columns=["frequency"])
df.index.name = "label"
df = df.sort_values("frequency", ascending=False)

# Plot the label frequencies
plt.rcParams["figure.figsize"] = (10, 6)
ax = df.plot(kind="bar", rot=0, legend=False, title="Label Frequencies in Training Set")
ax.set_xlabel("Labels")
ax.set_ylabel("Frequency")
plt.show()

# Dataset Preparation

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, dev_texts, train_labels, dev_labels = train_test_split(train_texts,
                                                                    train_labels,
                                                                    test_size=0.1,
                                                                    shuffle=True,
                                                                    random_state=1)

print("Train:", len(train_texts))
print("Dev:", len(dev_texts))
print("Test:", len(test_texts))

In [ ]:
class ClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

In [ ]:
# Metrics for Evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Full Finetuning with BERT models

In [ ]:
model_ids = ["prajjwal1/bert-tiny", "prajjwal1/bert-small", "google-bert/bert-base-uncased"]

accuracies = []
for model_id in model_ids:
    print(f"*** {model_id} ***")

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    num_labels = len(set(train_labels))
    model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=num_labels)

    total_params = sum(p.numel() for p in model.parameters())
    print(total_params)

    # Convert lists back to PyTorch tensors for the model
    train_labels = torch.tensor(train_labels)
    dev_labels = torch.tensor(dev_labels)
    test_labels = torch.tensor(test_labels)

    print(f"Train labels: {set(train_labels.tolist())}")
    print(f"Dev labels: {set(dev_labels.tolist())}")
    print(f"Test labels: {set(test_labels.tolist())}")


    train_texts_encoded = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
    dev_texts_encoded = tokenizer(dev_texts, padding=True, truncation=True, return_tensors="pt")
    test_texts_encoded = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

    train_dataset = ClassificationDataset(train_texts_encoded, train_labels)
    dev_dataset = ClassificationDataset(dev_texts_encoded, dev_labels)
    test_dataset = ClassificationDataset(test_texts_encoded, test_labels)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=2,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=int(len(train_dataset) / 16),
        weight_decay=0.01,
        logging_dir='./logs',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        no_cuda=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
    )

    trainer.train()
    test_results = trainer.evaluate(test_dataset)

    trainer.save_model(f"{os.getcwd()}/models/{model_id}")

    accuracies.append(test_results["eval_accuracy"])
    print(f"Model {model_id} achieved accuracy: {test_results['eval_accuracy']}")

# PEFT with LoRA

In [ ]:
#### Training with PEFT
model_ids = ["prajjwal1/bert-tiny", "prajjwal1/bert-small", "google-bert/bert-base-uncased"] #"prajjwal1/bert-tiny"

accuracies = []
for model_id in model_ids:
    print(f"*** {model_id} ***")

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    num_labels = len(set(train_labels))
    base_model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=num_labels)

    # **Step 1: Configure PEFT (LoRA)**
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,  # Sequence classification task
        inference_mode=False,       # Fine-tuning mode
        r=64,                        # LoRA rank
        lora_alpha=64,              # LoRA scaling factor
        lora_dropout=0.2            # Dropout for LoRA layers
    )

    # **Step 2: Wrap the Base Model with PEFT**
    model = get_peft_model(base_model, peft_config)
    model.print_trainable_parameters()  # Print trainable parameters for verification

    # Label mapping and preparation remain unchanged
    train_labels = [int(label.item()) if isinstance(label, torch.Tensor) else int(label) for label in train_labels]
    dev_labels = [int(label.item()) if isinstance(label, torch.Tensor) else int(label) for label in dev_labels]
    test_labels = [int(label.item()) if isinstance(label, torch.Tensor) else int(label) for label in test_labels]

    # label_mapping = {label: idx for idx, label in enumerate(sorted(set(train_labels)))}
    # train_labels = [label_mapping[label] for label in train_labels]
    # dev_labels = [label_mapping[label] for label in dev_labels]
    # test_labels = [label_mapping[label] for label in test_labels]

    train_labels = torch.tensor(train_labels)
    dev_labels = torch.tensor(dev_labels)
    test_labels = torch.tensor(test_labels)

    train_texts_encoded = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
    dev_texts_encoded = tokenizer(dev_texts, padding=True, truncation=True, return_tensors="pt")
    test_texts_encoded = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

    train_dataset = ClassificationDataset(train_texts_encoded, train_labels)
    dev_dataset = ClassificationDataset(dev_texts_encoded, dev_labels)
    test_dataset = ClassificationDataset(test_texts_encoded, test_labels)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=int(len(train_dataset) / 16),
        weight_decay=0.01,
        logging_dir='./logs',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        no_cuda=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
    )

    # **Step 3: Train with PEFT**
    trainer.train()

    # Evaluate the model
    test_results = trainer.evaluate(test_dataset)

    # Save model
    trainer.save_model(f"{os.getcwd()}/models/peft/{model_id}")

    accuracies.append(test_results["eval_accuracy"])
    print(f"Model {model_id} achieved accuracy: {test_results['eval_accuracy']}")

## Classification using the Fully Fine-Tuned model

In [ ]:
## Python Function to help in the intent_classification based on a given prompt
def classify_intent(query, tokenizer, model):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    print(logits)
    print(logits.size())
    predicted_label = torch.argmax(logits, dim=1).item()

    return predicted_label

def generate_prompt(verbalizer, input_sentence, intent_label) -> str:
  return f"{verbalizer}Input Sentence: {input_sentence}\nIntent Label: {intent_label}\nOutput:"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("models/google-bert/bert-base-uncased")

query = "How windy is it in Mumbai today?"
predicted_label = classify_intent(query, tokenizer, model)
print(f"Predicted Label: {predicted_label}")

# Task 2: From classified intent, we will generate JSON output for that tool’s API call (Few shot prompting with json templates for api calls using GPT-4o)


In [ ]:
verbalizer = """Based on the intent label (0, 1, 2), return the specific output as JSON. Below are examples:

Example 1:
Input Sentence: "Get me a cab from CMU to UPitt for 3 people at 8.30pm."
Intent Label: 0
Output:
{
   "from": "CMU",               # Extract "from" location from the input sentence if given else return empty string ""
   "to": "UPitt",               # Extract "to" location from the input sentence if given else return empty string ""
   "time": "8.30pm",            # Extract time, if not found return empty string ""
   "number_of_people": 3,       # Extract number of people, if not found return 1
}

Example 2:
Input Sentence: "Get me a cab to Target."
Intent Label - 0
Output:
{
  "from": "",
  "to": "Target",
  "time": "",
  "number_of_people": 1
}

Example 3:
Input Sentence: "Will it rain tomorrow in London?"
Intent Label - 1
Output:
{
  "location": "London",               # Extract location, if not found return "current_location"
  "forecast_days": 2                  # Extract number of days, if not found return 1
}

Example 4:
Input Sentence: "How is the weather outside?"
Intent Label - 1
Output:
{
  "location": "current_location",
  "forecast_days": 1
}

Example 5:
Input Sentence: "Is it raining in New York?"
Intent Label - 1
Output:
{
  "location": "New York",
  "forecast_days": 1
}

Example 6:
Input Sentence: "Book a table for today's lunch at Eggy's Diner for 3 people"
Intent Label - 2
Output:
{
  "date": today_date,          # Extract date like today's or tomorrow's date or any other day in the week's date, if not found return empty string ""
  "time": "",                  # Extract time, if not found return empty string ""
  "type": "lunch"              # Extract type, if not found return empty string ""
  "location": "Eggy's Diner",  # Extract location, if not found return empty string ""
  "number_of_people": 3        # Extract number of people, if not found return 1
}

Example 7:
Input Sentence: "Book a table at Joan's on Third for my family reunion on Saturday"
Intent Label - 2
Output:
{
  "date": Saturday_date,
  "time": "",
  "type": "family reunion"
  "location": "Joan's on Third",
  "number_of_people": 1
}


Example 8:
Input Sentence: "Book a restaurant for 6 near my morning meeting tomorrow at 1pm"
Intent Label - 2
Output:
{
  "date": tomorrow_date,
  "time": "1 pm",
  "type": ""
  "location": "near my morning meeting",
  "number_of_people": 6
}


Example 9:
Input Sentence: "Book a table at a restaurant near Times Square for 2 people tomorrow night"
Intent Label - 2
Output:
{
  "date": tomorrow_date,
  "time": "night",
  "type": ""
  "location": "near Times Square",
  "number_of_people": 2
}


End of Examples.

Now learning from the above examples, I want you to generate the output JSON for the given Input Sentence and Intent Label.
"""

In [ ]:
input_prompt = generate_prompt(verbalizer, query, predicted_label)

In [ ]:
client = openai.OpenAI(
  api_key = "",
  base_url = "",
)

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a helpful assistant that helps me generate JSON from a given prompt based on examples given!"},
    {"role": "user", "content": f"{input_prompt}"},
  ]
)

print(response.choices[0].message.content)

In [ ]:
json_ = response.choices[0].message.content

json_string = re.sub(r'json', '', json_).strip()
json_string = re.sub(r'`|\s+', ' ', json_string).strip()

input_json = json.loads(json_string)
# print(input_json)

## Integrating with Tools
1. [Weather API](https://www.weatherapi.com/docs/)

In [ ]:
def get_weather_or_forecast(api_key, location, forecast_days=1):
    if forecast_days > 1:
        url = f"https://api.weatherapi.com/v1/forecast.json?key={api_key}&q={location}&days={forecast_days}"
    else:
        url = f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={location}"

    response = requests.get(url)

    if response.status_code == 200:
        weather_data = response.json()

        if forecast_days > 1:
            forecast = []
            for day in weather_data["forecast"]["forecastday"]:
                forecast.append({
                    "date": day["date"],
                    "temperature_c": {
                        "max": day["day"]["maxtemp_c"],
                        "min": day["day"]["mintemp_c"]
                    },
                    "temperature_f": {
                        "max": day["day"]["maxtemp_f"],
                        "min": day["day"]["mintemp_f"]
                    },
                    "condition": day["day"]["condition"]["text"]
                })
            return {
                "location": weather_data["location"]["name"],
                "region": weather_data["location"]["region"],
                "country": weather_data["location"]["country"],
                "forecast_days": forecast
            }
        else:
            return {
                "temperature_c": weather_data["current"]["temp_c"],
                "temperature_f": weather_data["current"]["temp_f"],
                "condition": weather_data["current"]["condition"]["text"],
                "location": weather_data["location"]["name"],
                "region": weather_data["location"]["region"],
                "country": weather_data["location"]["country"]
            }
    else:
        print("Error fetching weather/forecast:", response.text)
        return None
    
def get_current_location():
    url = "https://ipinfo.io/json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        location = data.get("city", "unknown")
        return location
    else:
        print("Error fetching location:", response.text)
        return "unknown"

In [ ]:
WEATHER_API_KEY = "Put Your API key"

location = None
if input_json["location"] == "current_location":
  location = get_current_location()
  input_json["location"] = location
else:
  location = input_json["location"]

print(location)

dt = datetime.now()
eastern = timezone('US/Eastern')
current_time = dt.astimezone(eastern)
input_json["time"] = current_time.strftime("%Y-%m-%d %H:%M:%S")

print(input_json)

result = get_weather_or_forecast(WEATHER_API_KEY, input_json["location"], input_json["forecast_days"])
print(result)